In [1]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


from bs4 import BeautifulSoup





Libraries imported.


In [3]:

res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
df[0]
canada_data=df[0]
canada_data.columns=['PostalCode','Borough','Neighborhood']
canada_data=canada_data[canada_data.Borough != 'Not assigned'] #  Ignore cells with a borough that is Not assigned
# More than one neighborhood can exist in one postal code area separated with a comma
canada_data_gb =canada_data.groupby(['PostalCode','Borough'])['Neighborhood'].agg([('Neighborhood', ', '.join)]).reset_index()
canada_data_gb

#cleaning up not assigned 
for idx, row in canada_data.iterrows():
    if  canada_data.loc[idx,'Neighborhood'] == "Not assigned"  :
        canada_data.loc[idx,'Neighborhood'] = canada_data.loc[idx,'Borough']
        break
    else :
         canada_data.loc[idx,'Neighborhood'] = canada_data.loc[idx,'Neighborhood']
canada_data


#acessing dataset - stores in local computer  
df = pd.read_csv("Geospatial_Coordinates.csv")

df.columns=['PostalCode','Latitude','Longitude']
df.head(10)

Toronto_GeoCoder = pd.merge(canada_data_gb,
                 df[['PostalCode','Latitude', 'Longitude']],
                 on='PostalCode')
Toronto_GeoCoder.head()


/Users/keerthanamadhavan/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/Users/keerthanamadhavan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [4]:
CLIENT_ID = 'DWWOIW0WL2EK5LWMEVWHN5QZWVTVJHFILOR52RESUM35K5I1' # your Foursquare ID
CLIENT_SECRET = 'JG0FWBBLEXOFJRPLS4GTYL4SLKTNPXDFB5DYTSEPDPTVMNEP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: DWWOIW0WL2EK5LWMEVWHN5QZWVTVJHFILOR52RESUM35K5I1
CLIENT_SECRET:JG0FWBBLEXOFJRPLS4GTYL4SLKTNPXDFB5DYTSEPDPTVMNEP


In [5]:
address = 'East Toronto, Toronto'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 43.6534817, -79.3839347.


In [6]:
toronto_b_data=Toronto_GeoCoder[Toronto_GeoCoder['Borough'].str.contains("Toronto")].reset_index(drop=True)                      

toronto_b_data.loc[0, 'Neighborhood']

'The Beaches'

In [7]:
from pandas.io.json import json_normalize 

East_Toronto_data = toronto_b_data[toronto_b_data['Borough'] == 'East Toronto'].reset_index(drop=True)
East_Toronto_data.head()
East_Toronto_data.loc[0, 'Neighborhood']

# Get the neighborhood's latitude and longitude values
neighborhood_latitude = East_Toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = East_Toronto_data.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = East_Toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

# Now, let's get the top 100 venues for a sample site

LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

# Send the GET request and examine the resutls

results = requests.get(url).json()
results

# function that extracts the category of the venue

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
# Now we are ready to clean the json and structure it into a pandas dataframe.

venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()
# And how many venues were returned by Foursquare?


print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))


Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.
5 venues were returned by Foursquare.


/Users/keerthanamadhavan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:51: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


In [8]:
# Explore Neighborhoods in East Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL, agian
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
# create a new dataframe called East_Toronto_venues

East_Toronto_venues = getNearbyVenues(names=East_Toronto_data['Neighborhood'],
                                   latitudes=East_Toronto_data['Latitude'],
                                   longitudes=East_Toronto_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Business reply mail Processing Centre, South Central Letter Processing Plant Toronto


In [10]:
# check the size of the resulting dataframe
print(East_Toronto_venues.shape)
East_Toronto_venues.head()


# check how many venues were returned for each neighborhood
East_Toronto_venues.groupby('Neighborhood').count()

#  Analyze Each Neighborhood
East_Toronto_onehot  = pd.get_dummies(East_Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
East_Toronto_onehot['Neighborhood'] = East_Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [East_Toronto_onehot.columns[-1]] + list(East_Toronto_onehot.columns[:-1])
East_Toronto_onehot = East_Toronto_onehot[fixed_columns]
East_Toronto_onehot.head()

East_Toronto_grouped = East_Toronto_onehot.groupby('Neighborhood').mean().reset_index()
East_Toronto_grouped

num_top_venues = 5
?
for hood in East_Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = East_Toronto_grouped[East_Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

# create the df

#in desc order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# let's create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# Create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# Create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = East_Toronto_grouped['Neighborhood']

for ind in np.arange(East_Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(East_Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

East_Toronto_grouped_clustering = East_Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(East_Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

East_Toronto_merged = East_Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
East_Toronto_merged = East_Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

East_Toronto_merged.head() # check the last columns!

# let's visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set up the map strucs
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(East_Toronto_merged['Latitude'], East_Toronto_merged['Longitude'], East_Toronto_merged['Neighborhood'], East_Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters



(132, 7)
----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0  Light Rail Station  0.11
1         Pizza Place  0.05
2          Smoke Shop  0.05
3       Garden Center  0.05
4              Garden  0.05


----India Bazaar, The Beaches West----
                  venue  freq
0                  Park  0.09
1  Fast Food Restaurant  0.09
2           Pizza Place  0.09
3        Sandwich Place  0.09
4          Liquor Store  0.04


----Studio District----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.07
2            Gastropub  0.05
3               Bakery  0.05
4  American Restaurant  0.05


----The Beaches----
               venue  freq
0   Asian Restaurant   0.2
1              Trail   0.2
2  Health Food Store   0.2
3                Pub   0.2
4        Yoga Studio   0.0


----The Danforth West, Riverdale----
                venue  freq
0    Greek Restaurant  0.19
1  Italian Restaurant  0.07
2 

In [11]:
# you can examine each cluster and determine the discriminating venue categories that distinguish each cluster.

# Cluster 1

East_Toronto_merged.loc[East_Toronto_merged['Cluster Labels'] == 0, East_Toronto_merged.columns[[1] + list(range(5, East_Toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Bookstore,Restaurant,Yoga Studio,Café,Indian Restaurant


In [12]:
# Cluster 2
East_Toronto_merged.loc[East_Toronto_merged['Cluster Labels'] == 1, East_Toronto_merged.columns[[1] + list(range(5, East_Toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,East Toronto,1,Pizza Place,Fast Food Restaurant,Sandwich Place,Park,Burrito Place,Fish & Chips Shop,Gym,Ice Cream Shop,Intersection,Italian Restaurant


In [13]:
# Cluster 3
East_Toronto_merged.loc[East_Toronto_merged['Cluster Labels'] == 2, East_Toronto_merged.columns[[1] + list(range(5, East_Toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,2,Trail,Asian Restaurant,Pub,Health Food Store,Fast Food Restaurant,Cosmetics Shop,Coworking Space,Dessert Shop,Diner,Farmers Market


In [14]:
# Cluster 4
East_Toronto_merged.loc[East_Toronto_merged['Cluster Labels'] == 3, East_Toronto_merged.columns[[1] + list(range(5, East_Toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,3,Light Rail Station,Farmers Market,Burrito Place,Gym / Fitness Center,Park,Brewery,Pizza Place,Comic Shop,Recording Studio,Restaurant


Plotted the map using K means for the East Toronto from Toronto neighborhood.
